# Imports

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import string
import re
# !pip install Bio
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'Bio'

# Loading files

In [2]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

### Loading preprocessed data
Since data preprocessing takes time we have done some preprocessing and store the preprocessed data.

Those preprocessing are:
- Characters to ord numbers
- Bio sequency parameters (molecular_weight,	gravity,	iso_electric_point,	instability_index,	molar_extinction_coefficient,	secondary_structure_fraction)

In [3]:
X_train_preprocessed=pd.read_csv('./data/train_data_preprocessing1.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test_preprocessed=pd.read_csv('./data/test_data_preprocessing1.csv', sep=',')

In [4]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)


# Data preprocessing

### 1. Converting characters to numbers

In [5]:
# Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))
# for i in range(101):
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])

### 2. Getting DNA parameters by using bio-sequence

In [6]:
def get_DNA_parameters(data):
    """
    This function takes a dataframe that containes a column(seq) of DNA sequences
    It computer parameters related to all sequence and append those features to the input datafram
    @ input : DataFrame
    @ output : DataFrame
    """
    
    data = data
    cols = ['molecular_weight','gravity','iso_electric_point',
            'instability_index','molar_extinction_coefficient',
           'secondary_structure_fraction']

    for name in cols:
        data[name] = None

    for ind in range(len(data)):
        seq = data.iloc[ind]['seq']
        seq = ProteinAnalysis(seq)
        data[cols[0]][ind] = seq.molecular_weight()
        data[cols[1]][ind] = seq.gravy()
        data[cols[2]][ind] = seq.isoelectric_point()
        data[cols[3]][ind] = seq.instability_index()
        data[cols[4]][ind] = np.mean(seq.molar_extinction_coefficient())
        data[cols[5]][ind] = np.mean(seq.secondary_structure_fraction())
    
    return data

In [7]:
# X_test_ = get_DNA_parameters(X_test)
# X_train_ = get_DNA_parameters(X_train)

### 3. function to convert a DNA sequence string to a numpy array

In [8]:
# converts to lower case, changes any non 'acgt' characters to 'n'
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

### 4. DNA sequence string as an ordinal vector

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))
def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    return float_encoded

### 5. Function to one-hot encode a DNA sequence string

In [10]:
def one_hot_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int, n_values=5)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded

In [11]:
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

### 6. Afunction to give required representations

In [23]:
def DNA_represent(data,mode = 'ordinal',k=7):
    data = data
    cols = ['word'+str(i) for i in range(101)]
    if mode == 'spectrum':
        d = 101 - k + 1
        cols = ['seq'+str(i) for i in range(d)]
    
    X = []
    x = []
    bow = {}
    for ind in range(len(data)):
        seq = data.iloc[ind]['seq']

        if mode == 'ordinal':
            seq = string_to_array(seq)
            seq_arr = ordinal_encoder(seq)
       
        if mode == 'onehot':
            seq = string_to_array(seq)
            seq_arr = one_hot_encoder(seq)
            
        if mode == 'spectrum':
            seq_arr = getKmers(seq,k)
            for key in seq_arr:
                if key in bow.keys():
                    bow[key] += 1
                else:
                    bow[key] = 1
            
        X.append(seq_arr)
        x.append(seq_arr)
    X = np.array(X)
    X = pd.DataFrame(data=X,columns=cols)
    X = pd.concat([data,X],axis=1,sort=False)
    return X, np.array(x), bow
        

In [24]:
# X_train_,X ,_= DNA_represent(X_train,mode='ordinal')
# X_test_,X_t,_ = DNA_represent(X_test)

In [25]:
# X_train_words, _, bow_train= DNA_represent(X_train,mode='spectrum')
# X_test_words, _, bow_test = DNA_represent(X_test,mode='spectrum')

In [26]:
#bow_train

In [27]:
def BoW_data(data):
    _, _, bow_train= DNA_represent(X_train,mode='spectrum')
    _, data, _= DNA_represent(data,mode='spectrum')
    out = []
    for ind in data:
        temp = []
        for word in ind:
            if word in bow_train.keys():
                temp.append(bow_train[word])
            else:
                temp.append(0)
        out.append(temp)
    return np.array(out)

### Try to look at X and X_t in the following cells

In [28]:
X = BoW_data(X_train)

In [29]:
X_t = BoW_data(X_test)

In [30]:
X.shape

(2000, 95)

In [31]:
X_t.shape

(1000, 95)

### Preprocessing the target, if you want to use SVM uncomment the lines commented to make y{1,-1}

In [90]:
y = Y_train.drop(['Id'],axis = 1)
y = y['Bound']
y = np.array(y)
# for i in range(len(y)):
#     if y[i] == 0:
#         y[i] = -1

# The end ______ The end _______The end______the end

In [40]:
#BoW_data(X_test)

In [16]:
X_train_words = X_train_words.drop(['Id','seq'], axis=1)
X_test_words = X_test_words.drop(['Id','seq'], axis=1)

In [17]:
y = Y_train.drop(['Id'],axis = 1)

In [212]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

X_cross = onehot_encoder.fit_transform(X_train_words)
X_t_enc = onehot_encoder.fit_transform(X_test_words)



In [213]:
X_cross.shape

(2000, 121062)

In [215]:
X_t_enc.shape

(1000, 71373)

In [220]:
X_test_words.shape

(1000, 96)

In [219]:
X_train_words.shape

(2000, 96)

In [216]:
train_test_words  = [X_train_words,X_test_words]
train_test_words = pd.concat(train_test_words)

In [217]:
train_test_onE_hot = onehot_encoder.fit_transform(train_test_words)

In [218]:
X_cross = train_test_onE_hot[:2000,:]
X_t_enc = train_test_onE_hot[2000:,:]

In [223]:
X_cross.shape

(2000, 158293)

In [224]:
X_t_enc.shape

(1000, 158293)

# Feature selection

In [303]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, chi2

In [304]:
def feature_selector(X):
    #sel = VarianceThreshold()
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(X)
    return X
def best_feature_selector(X,y,num_features=50):
    #print(X.shape)
    features = SelectKBest(chi2, k=num_features).fit(X, y)
    #print(X_new.shape)
    return features

In [244]:
X = X_train.drop(['seq', 'Id'], axis=1)
X_t = X_test.drop(['seq', 'Id'], axis=1)
y = Y_train.drop(['Id'],axis = 1)
#trans = best_feature_selector(X,y,80)

In [143]:
X = trans.transform(X)
X_t = trans.transform(X_t)

In [270]:
y = np.array(y)

In [271]:
for i in range(len(y)):
    if y[i] == 0:
        y[i] = -1

# SVM model

In [53]:
from numpy import linalg
import cvxopt
import cvxopt.solvers
             
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

class SVM(object):

    def __init__(self, kernel=polynomial_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = y.reshape(1,n_samples)
        A = A.astype('float')
        A = cvxopt.matrix(A)
        #print(type(A))
        
        b = cvxopt.matrix(0.0)

        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

        # solve QP problem
        print('P',P.size,'q',q.size,'G',G.size,'h',h.size,'A',A.size,'b',b.size)
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        a = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))


In [54]:
def test_soft(X_train, y_train,X_test, y_test):
    
    clf = SVM(C=1000.1)
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    correct = np.sum(y_predict == y_test)
    print("%d out of %d predictions correct" % (correct, len(y_predict)))
    print ('the accuracy is: ',correct /len(y_predict) )
    return clf
    #plot_contour(X_train[y_train==1], X_train[y_train==-1], clf)


In [315]:
# X_cross
# X_t_enc

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [58]:
#model = test_soft(X_train, y_train,X_test, y_test)

In [336]:
preds=model.predict(X_t)

In [338]:
outs=[]
for i in range(len(preds)):
    if preds[i]==-1:
        outs.append(0)
    else:outs.append(1)

In [340]:
np.array(outs)

array([1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,

## Models

In [59]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    



# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)


## Kernel

In [60]:
X_train.std().mean()

55.46359572663006

In [61]:
import ipdb

In [62]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    
    def K(self, x, x_prime):
        return (x.T.dot(x_prime))**2
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
#         ipdb.set_trace()
#         A = X.T.dot(X) + np.eye(p)*lam*n
#         A = (X.T.dot((X.dot(X.T) + 1 )**2) + np.eye(n)*lam*n

#         b = y
        
        K = np.exp(-(1/(2*(6.995879868253351)))*np.linalg.norm(X-X)**2)
        
#         K = (X.dot(X.T)+1)**(300)
        
        self.beta = (X.T.dot(\
                             np.linalg.inv(K + np.eye(n)*lam*n))\
                             .dot(y))
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Cross Validation

In [63]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


In [94]:
kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y#.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

X_cross = onehot_encoder.fit_transform(X)
X_t_enc = onehot_encoder.fit_transform(X_t)


# scaler = StandardScaler()#MinMaxScaler() # StandardScaler()
# scaler.fit(X_cross)

# X_cross = scaler.transform(X_cross)

In [87]:
X_t_enc#.shape

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [92]:
from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score, roc_auc_score


X_cross = X
X_t_enc = X_t
y_cross = y

In [95]:
# models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
#           solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

models = {ksolveRR : 'Kernal Ridge Regression'}
for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_train)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if model ==solveWRR:
            w = np.random.rand(len(y_train))
            model_curr = solveWRR(X_train, y_train, w, lam=0.001)
        else:
            model_curr = model(X_train, y_train, lam=0.001)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.53125
accurracy fold 1: 0.5078125
accurracy fold 2: 0.578125
accurracy fold 3: 0.4375
accurracy fold 4: 0.51953125

Average accuracy Kernal Ridge Regression is : 0.51484375



In [96]:
models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
          solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

# models = {ksolveRR : 'Kernal Ridge Regression'}
for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_train)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if model ==solveWRR:
            w = np.random.rand(len(y_train))
            model_curr = solveWRR(X_train, y_train, w, lam=0.0001)
        else:
            model_curr = model(X_train, y_train, lam=0.0001)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.5024390243902439
accurracy fold 1: 0.5414634146341464
accurracy fold 2: 0.5365853658536586
accurracy fold 3: 0.5609756097560976
accurracy fold 4: 0.5686274509803921

Average accuracy Ridge Regression (RR) is : 0.5420181731229077

accurracy fold 0: 0.5182926829268293
accurracy fold 1: 0.5304878048780488
accurracy fold 2: 0.47560975609756095
accurracy fold 3: 0.5304878048780488
accurracy fold 4: 0.5609756097560976

Average accuracy Weighted Ridge Regression (WRR) is : 0.5231707317073171

accurracy fold 0: 0.5757575757575758
accurracy fold 1: 0.5343511450381679
accurracy fold 2: 0.48091603053435117


KeyboardInterrupt: 

In [116]:
# Cehckinf full model
model = ksolveRR(X_cross, y_cross, lam=0.0001)
model.fit()

model.Accuracy_check(X_cross, y_cross, threshold=0.5)

0.9555

# Predictions

In [117]:
model = ksolveRR(X_cross, y_cross, lam=0.0001)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

In [118]:
X_t_enc.shape

(1000, 158293)

In [119]:
model.beta.shape

(158293, 1)

In [120]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


In [121]:
sample['Bound'] = y_pred

In [122]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,0
998,998,1
999,999,1


In [123]:
#sample.tail()

In [124]:
sample.to_csv('./ksolveRR_65_cv_words.csv', index=False)